In [51]:
import pandas as pd
import numpy as np
from math import log2
from collections import Counter
from pprint import pprint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

In [52]:
def entropy(series):
    vals, counts = np.unique(series, return_counts=True)
    probs = counts / counts.sum()
    # handle single-class (entropy 0)
    return -np.sum([p * log2(p) for p in probs if p > 0])

def info_gain(data, split_attr, target_attr):
    total_entropy = entropy(data[target_attr])
    values, counts = np.unique(data[split_attr], return_counts=True)
    weighted_ent = 0.0
    for v, cnt in zip(values, counts):
        subset = data[data[split_attr] == v]
        weighted_ent += (cnt / counts.sum()) * entropy(subset[target_attr])
    return total_entropy - weighted_ent

def split_info(data, split_attr):
    values, counts = np.unique(data[split_attr], return_counts=True)
    probs = counts / counts.sum()
    return -np.sum([p * log2(p) for p in probs if p > 0])

def majority_class(series):
    vals, counts = np.unique(series, return_counts=True)
    return vals[np.argmax(counts)]

In [53]:
def id3(data, original_data, features, target_attr, parent_class=None):
    # If all targets same -> leaf
    if len(np.unique(data[target_attr])) == 1:
        return np.unique(data[target_attr])[0]
    # If no data -> return majority of original
    if data.shape[0] == 0:
        return majority_class(original_data[target_attr])
    # If no features -> return parent majority
    if len(features) == 0:
        return parent_class

    parent_class = majority_class(data[target_attr])

    # compute info gain for each feature
    gains = [(feature, info_gain(data, feature, target_attr)) for feature in features]
    # choose best feature (highest gain)
    best_feature, best_gain = max(gains, key=lambda x: x[1])

    tree = {best_feature: {}}
    for val in np.unique(data[best_feature]):
        sub_data = data[data[best_feature] == val]
        sub_features = [f for f in features if f != best_feature]
        subtree = id3(sub_data, data, sub_features, target_attr, parent_class)
        tree[best_feature][val] = subtree
    return tree


In [54]:
def c45(data, original_data, features, target_attr, parent_class=None):
    # If all targets same -> leaf
    if len(np.unique(data[target_attr])) == 1:
        return np.unique(data[target_attr])[0]
    # If no data -> return majority of original
    if data.shape[0] == 0:
        return majority_class(original_data[target_attr])
    # If no features -> return parent majority
    if len(features) == 0:
        return parent_class

    parent_class = majority_class(data[target_attr])

    # compute gain ratio for each feature
    gain_ratios = []
    for feature in features:
        ig = info_gain(data, feature, target_attr)
        si = split_info(data, feature)
        gr = ig / si if si > 0 else 0
        gain_ratios.append((feature, gr))

    best_feature, best_gr = max(gain_ratios, key=lambda x: x[1])

    tree = {best_feature: {}}
    for val in np.unique(data[best_feature]):
        sub_data = data[data[best_feature] == val]
        sub_features = [f for f in features if f != best_feature]
        subtree = c45(sub_data, data, sub_features, target_attr, parent_class)
        tree[best_feature][val] = subtree
    return tree


In [55]:
def collect_leaf_labels(subtree):
    """Return list of leaf labels found in subtree (recursive)."""
    leaves = []
    if not isinstance(subtree, dict):
        return [subtree]
    # subtree is dict with one key: attribute
    attr = list(subtree.keys())[0]
    branches = subtree[attr]
    for v in branches.values():
        if isinstance(v, dict):
            leaves.extend(collect_leaf_labels(v))
        else:
            leaves.append(v)
    return leaves

def predict(tree, sample):
    """Predict label for a single sample. If unseen branch, fallback to majority among descendant leaves."""
    if not isinstance(tree, dict):
        return tree
    attr = list(tree.keys())[0]
    branches = tree[attr]
    sample_val = sample[attr]
    # exact match
    if sample_val in branches:
        return predict(branches[sample_val], sample)
    # unseen branch: try to match by type conversions (int/str)
    # (cover case where keys might be ints and sample a numpy int, etc.)
    for key in branches:
        try:
            # try numeric equality
            if isinstance(key, (int, float)) and float(sample_val) == float(key):
                return predict(branches[key], sample)
        except Exception:
            pass
    # fallback: collect descendant leaves and return majority
    leaf_labels = collect_leaf_labels(tree)
    if leaf_labels:
        return Counter(leaf_labels).most_common(1)[0][0]
    # extreme fallback
    return None

In [56]:
df = pd.read_csv("Data/drug_200.csv")

# Convert categorical columns to strings (stable keys)
df["Sex"] = df["Sex"].astype(str)
df["BP"] = df["BP"].astype(str)
df["Cholesterol"] = df["Cholesterol"].astype(str)
df["Drug"] = df["Drug"].astype(str)

# Convert continuous Na_to_K into binary using median threshold
threshold = df["Na_to_K"].median()
df["Na_to_K_Binary"] = (df["Na_to_K"] > threshold).astype(int)

# Drop original continuous column
df = df.drop(columns=["Na_to_K"])

# Ensure columns are in deterministic order
feature_columns = ["Age", "Sex", "BP", "Cholesterol", "Na_to_K_Binary"]

df["Age"] = df["Age"].astype(str)



In [57]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

id3_metrics = []
c45_metrics = []

# We'll store the last fold trees for printing
last_id3_tree = None
last_c45_tree = None

for train_idx, test_idx in kf.split(df):
    train = df.iloc[train_idx].reset_index(drop=True)
    test = df.iloc[test_idx].reset_index(drop=True)

    features = feature_columns.copy()
    target = "Drug"

    # train trees
    id3_tree = id3(train, train, features, target)
    c45_tree = c45(train, train, features, target)

    last_id3_tree = id3_tree
    last_c45_tree = c45_tree

    # predict for test set
    y_true = test[target].values
    y_pred_id3 = [predict(id3_tree, row) for _, row in test.iterrows()]
    y_pred_c45 = [predict(c45_tree, row) for _, row in test.iterrows()]

    # compute weighted metrics (multiclass)
    for name, y_pred, store in [
        ("ID3", y_pred_id3, id3_metrics),
        ("C4.5", y_pred_c45, c45_metrics),
    ]:
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="weighted", zero_division=0)
        rec = recall_score(y_true, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
        store.append([acc, prec, rec, f1])



In [59]:
# average metrics across folds
def avg_metrics(metrics_list):
    return np.mean(np.array(metrics_list), axis=0)

id3_avg = avg_metrics(id3_metrics)
c45_avg = avg_metrics(c45_metrics)

print("=== ID3 Decision Tree (averaged over 5 folds) ===")
print(f"Accuracy: {id3_avg[0]:.4f}, Precision (weighted): {id3_avg[1]:.4f}, Recall (weighted): {id3_avg[2]:.4f}, F1 (weighted): {id3_avg[3]:.4f}")
print()
print("=== C4.5 Decision Tree (averaged over 5 folds) ===")
print(f"Accuracy: {c45_avg[0]:.4f}, Precision (weighted): {c45_avg[1]:.4f}, Recall (weighted): {c45_avg[2]:.4f}, F1 (weighted): {c45_avg[3]:.4f}")
print("\nSample ID3 tree (from last fold):")
pprint(last_id3_tree)
print("\nSample C4.5 tree (from last fold):")
pprint(last_c45_tree)

=== ID3 Decision Tree (averaged over 5 folds) ===
Accuracy: 0.4600, Precision (weighted): 0.4852, Recall (weighted): 0.4600, F1 (weighted): 0.4602

=== C4.5 Decision Tree (averaged over 5 folds) ===
Accuracy: 0.8700, Precision (weighted): 0.8595, Recall (weighted): 0.8700, F1 (weighted): 0.8557

Sample ID3 tree (from last fold):
{'Age': {'15': 'drugX',
         '16': 'drugY',
         '17': 'drugX',
         '18': {'BP': {'HIGH': 'drugY', 'NORMAL': 'drugX'}},
         '19': {'Cholesterol': {'HIGH': 'drugA', 'NORMAL': 'drugY'}},
         '20': {'BP': {'HIGH': {'Sex': {'F': 'drugA', 'M': 'drugY'}},
                       'LOW': 'drugX',
                       'NORMAL': 'drugX'}},
         '21': 'drugY',
         '22': {'BP': {'HIGH': 'drugY', 'NORMAL': 'drugX'}},
         '23': {'BP': {'HIGH': {'Sex': {'F': 'drugY', 'M': 'drugA'}},
                       'LOW': 'drugC',
                       'NORMAL': {'Cholesterol': {'HIGH': {'Na_to_K_Binary': {np.int64(0): 'drugX',
                   